<a href="https://colab.research.google.com/github/Mayank-Bansal24/LAPLSTM-Models/blob/main/default_weightst01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import losses as lo_ss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import string
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.utils import class_weight
from sklearn.metrics import classification_report , roc_auc_score
import keras 
import keras.backend as K
from google.colab import drive
from numpy import savetxt

In [ ]:
drive.mount('/content/gdrive')


In [9]:
t=1

In [10]:
import os
os.chdir('/content/gdrive/MyDrive/laplstm/default class weights/t'+str(t))

In [11]:

github_raw_link="https://raw.githubusercontent.com/Aryansh085/LAP/main/858417d1-7d54-4115-a01b-fdda5e03ada3_testing_combined_rows4035_disc_1_0p9_MULTIPLY_preproces_155cols.csv"
df1=pd.read_csv((github_raw_link),header = None)
dftemp=np.array(df1)
dftemp.shape
dftemp2=dftemp[:,5:]
dftemp2=np.array(dftemp2)
print(dftemp2.shape)

(4035, 150)


In [12]:
# dftemp2
dftemp2.shape
# dftemp2=pd.DataFrame(dftemp2)
# dftemp2.head()

(4035, 150)

--> muliply the reward of that instance into the loss of that instance to give weight

In [13]:
from sklearn.preprocessing import StandardScaler

slr=StandardScaler()
slr.fit(dftemp2)
dftemp2 = slr.transform(dftemp2)

In [14]:
# f1-score function
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
    

In [15]:
y=np.array(dftemp[:,0])
print(y)
y_new=y[t:]
y_new.shape


[2. 0. 0. ... 0. 2. 2.]


(4034,)

In [16]:
x=np.array(dftemp2)
# print(x)
# print(x.shape)
x.shape,y_new.shape

((4035, 150), (4034,))

In [17]:
x_3d=[]
for i in range(4035-t):
  tmp=[]
  for j in range(t):
    tmp.append(x[j+i])
  x_3d.append(tmp)
x_3d= np.array(x_3d)
x_3d.shape

(4034, 1, 150)

In [18]:
train_test_ratio=0.8
print(x_3d.shape[0])
x_train=[]
y_train=[]
x_test=[]
y_test=[]
for i in range(x_3d.shape[0]):
  l=[]
  ly=[]
  no=np.random.randint(100)
  if(no<80):
    l=x_3d[i]
    ly=y_new[i]
    x_train.append(l)
    y_train.append(ly)
  else:
    l=x_3d[i]
    ly=y_new[i]
    x_test.append(l)
    y_test.append(ly)


x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)
x_train.shape,y_train.shape,x_test.shape,y_test.shape

4034


((3188, 1, 150), (3188,), (846, 1, 150), (846,))

In [19]:
cnt0=0
cnt1=0
cnt2=0
for i in range(x_train.shape[0]):
  if(y_train[i]==0):
    cnt0+=1
  elif(y_train[i]==1):
    cnt1+=1
  elif(y_train[i]==2):
    cnt2+=1
print(cnt0)
print(cnt1)
print(cnt2)  

1448
835
905


In [20]:
sklearn_weights=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
sklearn_weights=dict(zip(np.unique(y_train),sklearn_weights))
sklearn_weights

{0.0: 0.7338858195211786, 1.0: 1.2726546906187626, 2.0: 1.1742173112338858}

In [21]:

# model = Sequential()
# model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=( t,150,)))
# model.add(LSTM(100, activation='relu', return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=True))
# model.add(LSTM(25, activation='relu'))
# model.add(Dense(20, activation='relu'))
# model.add(Dense(10, activation='relu'))
# model.add(Dense(3))
# model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=1)
# test_output = model.predict(x_test, verbose=1)
# # history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)
# # test_output = model.predict(x=, verbose=0)
# # print(test_output)

In [22]:
# sklearn_weights={0:1738,1:2348,2:2362}
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=( t,150,)))
model.add(LSTM(400, activation='relu', return_sequences=True))
model.add(LSTM(800, activation='relu', return_sequences=True))
model.add(LSTM(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3,activation='sigmoid'))
optimi=keras.optimizers.Adam()
model.compile(optimizer=optimi, loss='sparse_categorical_crossentropy',metrics=['accuracy',get_f1])

# history = model.fit(x_train, y_train, epochs=30, validation_split=0.2, verbose=1)
history = model.fit(x_train, y_train, epochs=25, validation_split=0.2, verbose=1,class_weight=sklearn_weights)
test_output = model.predict(x_test, verbose=1)


Epoch 1/25
80/80 [==============================] - 8s 25ms/step - loss: 1.0942 - accuracy: 0.4212 - get_f1: 1.1747 - val_loss: 1.1001 - val_accuracy: 0.2288 - val_get_f1: 1.1752
Epoch 2/25
80/80 [==============================] - 3s 37ms/step - loss: 1.0936 - accuracy: 0.3302 - get_f1: 1.0519 - val_loss: 1.0997 - val_accuracy: 0.3574 - val_get_f1: 1.2335
Epoch 3/25
80/80 [==============================] - 2s 22ms/step - loss: 1.0745 - accuracy: 0.4302 - get_f1: 1.1935 - val_loss: 1.1731 - val_accuracy: 0.3386 - val_get_f1: 1.2193
Epoch 4/25
80/80 [==============================] - 1s 17ms/step - loss: 1.0163 - accuracy: 0.5004 - get_f1: 1.1341 - val_loss: 1.2054 - val_accuracy: 0.3260 - val_get_f1: 1.2027
Epoch 5/25
80/80 [==============================] - 1s 17ms/step - loss: 0.8975 - accuracy: 0.5600 - get_f1: 1.1356 - val_loss: 1.5068 - val_accuracy: 0.3401 - val_get_f1: 1.1611
Epoch 6/25
80/80 [==============================] - 1s 17ms/step - loss: 0.7462 - accuracy: 0.6557 - get_

In [23]:
test_output_1=np.array([np.argmax(i)  for i in test_output])
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,test_output_1))
test_output=pd.DataFrame(test_output)
print(accuracy_score(y_test,test_output_1))


[[142 171  66]
 [ 71 107  42]
 [ 98 110  39]]
0.3404255319148936


In [24]:
print(get_f1(y_test.astype('float32'), test_output_1.astype('float32')))

tf.Tensor(0.59481037, shape=(), dtype=float32)


In [25]:
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
savetxt('arrxtrain.csv', x_train_reshaped, delimiter=',')
savetxt('arrytrain.csv', y_train, delimiter=',')
savetxt('arrxtest.csv', x_test_reshaped, delimiter=',')
savetxt('arrytest.csv', y_test, delimiter=',')